In [36]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

In [37]:
WINDOW_SIZE = 25
NUMBER_OF_FEATURES = 4
NUMBER_OF_CLASSES = 2

MOTION_CLASSES = ['STAND', 'STEPS']
MOTIONTYPE_CLASSES = ['SML', 'MED', 'LAR']
MOTIONSPEED_CLASSES = ['SLOW', 'AVERAGE', 'FAST']


# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)

DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

# numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TRAIN_CLASSIFY_MOTION_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTION-V0.npy'
ALL_Y_TRAIN_CLASSIFY_MOTION_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTION-V0.npy'

In [38]:
LABEL_TO_CATEGORY_MOTION = {label: category for category, label in enumerate(MOTION_CLASSES)}
LABEL_TO_CATEGORY_MOTIONTYPE = {label: category for category, label in enumerate(MOTIONTYPE_CLASSES)}
LABEL_TO_CATEGORY_MOTIONSPEED = {label: category for category, label in enumerate(MOTIONSPEED_CLASSES)}

print(LABEL_TO_CATEGORY_MOTION)
print(LABEL_TO_CATEGORY_MOTIONTYPE)
print(LABEL_TO_CATEGORY_MOTIONSPEED)

{'STAND': 0, 'STEPS': 1}
{'SML': 0, 'MED': 1, 'LAR': 2}
{'SLOW': 0, 'AVERAGE': 1, 'FAST': 2}


In [39]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TRAIN2-STAND1.xlsx', 'PROC-TRAIN2-STAND2.xlsx', 'PROC-TRAIN2-STAND3.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-100BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-110BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-112BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-124BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-164BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-196BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-200BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-224BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-31BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-41BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-49BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-50BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-56BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-60BPM.xlsx',

In [40]:
def normalizeSensorData(sensor_input):
    return (sensor_input + 180)/360


In [41]:
def df2Xy(df, windowSize=5):
  X = []
  y_Motion= []

  # NORMALIZE THE SENSOR DATA
  #df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalizeSensorData)
  #df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalizeSensorData)
  #df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalizeSensorData)
  #df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalizeSensorData)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch_Delta', 'L_Roll_Delta', 'R_Pitch_Delta', 'R_Roll_Delta']].values.tolist()

        # add row values to the input
        input_data_list.append(row_values)

    # turnn list into array to do arthimetic
    input_data_array = np.array(input_data_list)

    # remove all negative sinces we want to the total change (we dont care which direction)
    input_data_array = np.abs(input_data_array)

    # calculate the total change for each sennsor value (TOTAL POSITIVE SENSOR CHANGES)
    input_data_array = np.sum(input_data_array, axis=0)

    # turn back to list
    input_data_list = input_data_array.tolist()

    # add our input to our total inputs, marked as X
    X.append(input_data_list)

  y_Motion = df.iloc[windowSize - 1:, df.columns.get_loc('Class_Motion')].tolist()
  #y_MotionType = df.iloc[windowSize - 1:, df.columns.get_loc('Class_MotionType')].tolist()
  #y_MotionSpeed = df.iloc[windowSize - 1:, df.columns.get_loc('Class_MotionSpeed')].tolist()


  return (np.array(X), np.array(y_Motion))

In [42]:
ALL_X_TRAIN_MOTION = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN_MOTION = np.empty((0))  # List to store all Y training data

In [43]:
def processData(ALL_X_TRAIN_MOTION, ALL_Y_TRAIN_MOTION):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTrain, yTrain_Motion = df2Xy(df, WINDOW_SIZE)
        
        ALL_X_TRAIN_MOTION = np.concatenate([ALL_X_TRAIN_MOTION, xTrain])
        ALL_Y_TRAIN_MOTION = np.concatenate([ALL_Y_TRAIN_MOTION, yTrain_Motion])
        
        print('xTrain.shape:', xTrain.shape)
        print('ALL_X_TRAIN.shape:', ALL_X_TRAIN_MOTION.shape, ' ALL_Y_TRAIN_CURRVEL.shape:', ALL_Y_TRAIN_MOTION.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_motion_label = np.vectorize(LABEL_TO_CATEGORY_MOTION.get)(ALL_Y_TRAIN_MOTION)

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CLASSIFY_MOTION_PATH, ALL_X_TRAIN_MOTION)
    np.save(ALL_Y_TRAIN_CLASSIFY_MOTION_PATH, numerical_motion_label)

In [44]:
if not os.path.exists(ALL_X_TRAIN_CLASSIFY_MOTION_PATH):
    processData(ALL_X_TRAIN_MOTION, ALL_Y_TRAIN_MOTION)

ALL_X_TRAIN_CLASSIFY_MOTION = np.load(ALL_X_TRAIN_CLASSIFY_MOTION_PATH)
ALL_Y_TRAIN_CLASSIFY_MOTION = np.load(ALL_Y_TRAIN_CLASSIFY_MOTION_PATH)

print(ALL_X_TRAIN_CLASSIFY_MOTION.shape)
print(ALL_Y_TRAIN_CLASSIFY_MOTION.shape)

xTrain.shape: (1335, 4)
ALL_X_TRAIN.shape: (1335, 4)  ALL_Y_TRAIN_CURRVEL.shape: (1335,)
-------------------------------
xTrain.shape: (3288, 4)
ALL_X_TRAIN.shape: (4623, 4)  ALL_Y_TRAIN_CURRVEL.shape: (4623,)
-------------------------------
xTrain.shape: (1743, 4)
ALL_X_TRAIN.shape: (6366, 4)  ALL_Y_TRAIN_CURRVEL.shape: (6366,)
-------------------------------
xTrain.shape: (3381, 4)
ALL_X_TRAIN.shape: (9747, 4)  ALL_Y_TRAIN_CURRVEL.shape: (9747,)
-------------------------------
xTrain.shape: (3274, 4)
ALL_X_TRAIN.shape: (13021, 4)  ALL_Y_TRAIN_CURRVEL.shape: (13021,)
-------------------------------
xTrain.shape: (4560, 4)
ALL_X_TRAIN.shape: (17581, 4)  ALL_Y_TRAIN_CURRVEL.shape: (17581,)
-------------------------------
xTrain.shape: (3773, 4)
ALL_X_TRAIN.shape: (21354, 4)  ALL_Y_TRAIN_CURRVEL.shape: (21354,)
-------------------------------
xTrain.shape: (3181, 4)
ALL_X_TRAIN.shape: (24535, 4)  ALL_Y_TRAIN_CURRVEL.shape: (24535,)
-------------------------------
xTrain.shape: (3072, 4)
